In [142]:
# Packages for Web-Scraping
from requests import get
from bs4 import BeautifulSoup
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd

# Packages for Saving File after Scraping
import numpy as np

C:\Users\catat\AppData\Local\Temp\ipykernel_38056\2163921707.py:7: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [143]:
url = 'https://archiveofourown.org/works/search?commit=Search&page=1&work_search%5Bbookmarks_count%5D=&work_search%5Bcharacter_names%5D=&work_search%5Bcomments_count%5D=&work_search%5Bcomplete%5D=&work_search%5Bcreators%5D=&work_search%5Bcrossover%5D=&work_search%5Bfandom_names%5D=Good+Omens+%28TV%29%2CGood+Omens+-+Neil+Gaiman+%26+Terry+Pratchett&work_search%5Bfreeform_names%5D=&work_search%5Bhits%5D=&work_search%5Bkudos_count%5D=%26gt%3B100&work_search%5Blanguage_id%5D=&work_search%5Bquery%5D=&work_search%5Brating_ids%5D=&work_search%5Brelationship_names%5D=&work_search%5Brevised_at%5D=&work_search%5Bsingle_chapter%5D=0&work_search%5Bsort_column%5D=kudos_count&work_search%5Bsort_direction%5D=desc&work_search%5Btitle%5D=&work_search%5Bword_count%5D='
response = get(url)
print(response.text[:500])

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8"/>
    <meta http-equiv="x-ua-compatible" content="ie=edge"/>
    <meta name="keywords" content="fanfiction, transformative works, otw, fair use, archive"/>
    <meta name="language" content="en-US"/>
    <meta name="subject" content="fandom"/>
    <meta name="description" content="An Archive of Our Own, a project of the
    Organization for Transformative Works"/>
    <meta name="distribution" content="GLOBAL"/>
    <meta name="


In [144]:
print(len(response.text))

126524


In [145]:

# 1st page BeautifulSoup object, confirm type
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [146]:
# finding each row of data we want to scrape, confirm type, check length
statement_containers = html_soup.find_all('li')
print(type(statement_containers))
print(len(statement_containers))

<class 'bs4.element.ResultSet'>
450


In [147]:
fiction_works = []
for s in statement_containers:
    if s.get('role') != None:
        fiction_works.append(s)

In [148]:
print(len(fiction_works))

20


In [149]:
# get the title and the author

header = fiction_works[0].find('h4', class_ = 'heading')
header_elements = header.find_all('a')
title = header_elements[0].get_text(strip=True)
author = header_elements[1].get_text(strip=True)
print(title)
print(author)

In the (Second) Beginning
cherryfeather


In [150]:
fiction_items = []
i = 0
for container in statement_containers:
    i += 1
    element = container.find('li', role_= 'article')
    fiction_items.append(element)
    
print(len(fiction_items))
print(i)

450
450


In [151]:
#get the list of tags

tags = []
tag_list_container = fiction_works[0].find('h5', class_ = 'fandoms heading')
tag_list = tag_list_container.find_all('a', class_ = 'tag')
for tag_element in tag_list:
    tag = tag_element.get_text(strip=True)
    tags.append(tag)
    
print(tags)

['Good Omens (TV)', 'Good Omens - Neil Gaiman & Terry Pratchett']


In [152]:
# get the work summary

summary = ""

summary_container = fiction_works[0].find('blockquote', class_ = 'userstuff summary')
summary_paragraphs = summary_container.find_all('p')
for summary_paragraph in summary_paragraphs:
    phrase = summary_paragraph.get_text(strip=True)
    summary += phrase
    
print(summary)

As their lunch stretches on Aziraphale slowly comes to realize that Crowley is—enjoying him. Enjoying Aziraphale’s conversation, and company, far more openly than he has in most of Aziraphale’s memory. And Aziraphale knows that he himself is just chattering on, letting conversational tangents carry him along, and—it’s definitely relief, for him, knowing for the first time in a long time that they aren’t being watched, that no one is keeping score for now.-Aziraphale realizes that Crowley's been saying something rather loudly for a week.


In [153]:
# get the other pieces of information: no of tags, hits words etc.

stats_list = fiction_works[0].find('dl', class_ = 'stats')
word_count = stats_list.find('dd', class_ = 'words').get_text(strip=True)
chapter_count = stats_list.find('dd', class_ = 'chapters').get_text(strip=True).split("/")[0]

comment_count = stats_list.find('dd', class_ = 'comments').find('a').get_text(strip=True)
kudos_count = stats_list.find('dd', class_ = 'kudos').find('a').get_text(strip=True)
bookmarks_count = stats_list.find('dd', class_ = 'bookmarks').find('a').get_text(strip=True)
hits_count = stats_list.find('dd', class_ = 'hits').get_text(strip=True)

print("comments " + comment_count)
print("kudos " + kudos_count)
print("bookmarks " + bookmarks_count)
print("hits " + hits_count)

comments 565
kudos 25409
bookmarks 3999
hits 151598


In [154]:
def get_last_index(id_):
    url = "https://archiveofourown.org/works/" + str(id_) + "/kudos?before=4577931054&page=1"
    response = get(url)
    last_index_container = html_soup.find('ol', class_ = "pagination actions")
    index_list = last_index_container.find_all('li')
    last_index = index_list[-2].get_text(strip=True)
    return last_index

In [155]:
def get_kudos(id_):
    last_page_index = get_last_index(id_)
    kudos = [] 
    for i in range(1, int(last_page_index)):
        working_url = "https://archiveofourown.org/works/" + str(id_) + "/kudos?before=4577931054&page=" + str(i)
        response = get(working_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        kudos_container = html_soup.find('div', class_ = "kudos-index region")
        if type(kudos_container).__name__ == 'NoneType':
            continue
        kudos_div = kudos_container.find('p', class_="kudos")
        if type(kudos_div).__name__ != 'NoneType':
            kudos_people = kudos_div.find_all('a')
            for kudos_person in kudos_people:
                person = kudos_person.get_text(strip=True)
                kudos.append(person)
    
    return kudos

In [156]:
# group everything into a separate method

def get_info(fiction_work, titles, authors, tags, summaries, words, chapters, comments, kudos, bookmarks, hits, fiction_ids):
    fiction_id_string = fiction_work.get('id')
    fiction_id = fiction_id_string.split("_")[1]
    fiction_ids.append(int(fiction_id))
    header = fiction_work.find('h4', class_ = 'heading')
    header_elements = header.find_all('a')
    if type(header_elements).__name__ != 'NoneType':
        title = header_elements[0].get_text(strip=True)
        titles.append(str(title))
    else:
         titles.append(None)

    author = ""
    if (len(header_elements) > 1):
        author = header_elements[1].get_text(strip=True)
        authors.append(str(author))
    else:
        authors.append(None)
    
    tags_list = []
    tag_list_container = fiction_work.find('h5', class_ = 'fandoms heading') 
    tag_list = tag_list_container.find_all('a', class_ = 'tag')
    for tag_element in tag_list:
        tag = tag_element.get_text(strip=True)
        tags_list.append(tag)
    
    tags.append(tags_list)
        
    
    summary = ""
    summary_container = fiction_work.find('blockquote', class_ = 'userstuff summary')
    print(type(summary_container))
    if type(summary_container).__name__ != 'NoneType':
        summary_paragraphs = summary_container.find_all('p')
        for summary_paragraph in summary_paragraphs:
            phrase = summary_paragraph.get_text(strip=True) 
            summary += phrase
    summaries.append(str(summary))
    
    stats_list = fiction_work.find('dl', class_ = 'stats')
    if type(stats_list).__name__ != 'NoneType':
        word_count = stats_list.find('dd', class_ = 'words')
        if type(word_count).__name__ != 'NoneType':
            word_count = word_count.get_text(strip=True).replace(",", "" )
            words.append(float(word_count))
        else:
            words.append(0.0)
        
        chapter_count = stats_list.find('dd', class_ = 'chapters')
        if type(chapter_count).__name__ != 'NoneType':
            chapter_count = chapter_count.get_text(strip=True).split("/")[0]
            chapters.append(int(chapter_count))
        else:
            chapters.append(0)
        
        comment_count = stats_list.find('dd', class_ = 'comments')
        if type(comment_count).__name__ != 'NoneType':
            comment_count = comment_count.find('a')
            if type(comment_count).__name__ != 'NoneType':
                comment_count = comment_count.get_text(strip=True)
                comments.append(int(comment_count))
            else:
                comments.append(0)
        else:
            comments.append(0)
        
        kudos_count = stats_list.find('dd', class_ = 'kudos')
        if type(kudos_count).__name__ != 'NoneType':
            kudos_count = kudos_count.find('a')
            if type(kudos_count).__name__ != 'NoneType':
                kudos_count = kudos_count.get_text(strip=True)
                kudos.append(int(kudos_count))
            else:
                kudos.append(0)
        else:
            kudos.append(0)
        
        bookmarks_count = stats_list.find('dd', class_ = 'bookmarks')
        if type(bookmarks_count).__name__ != 'NoneType':
            bookmarks_count = bookmarks_count.find('a')
            if type(bookmarks_count).__name__ != 'NoneType':
                bookmarks_count = bookmarks_count.get_text(strip=True)
                bookmarks.append(int(bookmarks_count))
            else:
                bookmarks.append(0)
        else:
            bookmarks.append(0)
                
        hits_count = stats_list.find('dd', class_ = 'hits')
        if type(hits_count).__name__ != 'NoneType':
            hits_count = hits_count.get_text(strip=True)
            hits.append(int(hits_count))
        else:
            hits.append(0)
   

In [157]:
# get the info for all the fan-fic works on one page
ids = []
titles = []
authors = []
tags = []
summaries = [] 
words = [] 
chapters = [] 
comments = [] 
kudos = [] 
bookmarks = [] 
hits = []
fiction_ids = []
for fiction_work in fiction_works:
    get_info(fiction_work, titles, authors, tags, summaries, words, chapters, comments, kudos, bookmarks, hits, fiction_ids)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


In [158]:
# generalize and get the fan-fic works for 2 pages

url = "https://archiveofourown.org/works/search?commit=Search&page=" + str(2) + "&work_search%5Bbookmarks_count%5D=&work_search%5Bcharacter_names%5D=&work_search%5Bcomments_count%5D=&work_search%5Bcomplete%5D=&work_search%5Bcreators%5D=&work_search%5Bcrossover%5D=&work_search%5Bfandom_names%5D=Good+Omens+%28TV%29%2CGood+Omens+-+Neil+Gaiman+%26+Terry+Pratchett&work_search%5Bfreeform_names%5D=&work_search%5Bhits%5D=&work_search%5Bkudos_count%5D=%26gt%3B100&work_search%5Blanguage_id%5D=&work_search%5Bquery%5D=&work_search%5Brating_ids%5D=&work_search%5Brelationship_names%5D=&work_search%5Brevised_at%5D=&work_search%5Bsingle_chapter%5D=0&work_search%5Bsort_column%5D=kudos_count&work_search%5Bsort_direction%5D=desc&work_search%5Btitle%5D=&work_search%5Bword_count%5D="
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
statement_containers = html_soup.find_all('li')
fiction_works = []
for s in statement_containers:
    if s.get('role') != None:
        fiction_works.append(s)
        
for fiction_work in fiction_works:
    get_info(fiction_work, titles, authors, tags, summaries, words, chapters, comments, kudos, bookmarks, hits, fiction_ids)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


In [159]:
print(len(kudos))

40


In [160]:
print(kudos[20])

8135


In [161]:
# put the already obtained data in a dataframe
# make a dataframe
fanfic_test_df = pd.DataFrame(
    {'title': titles,
     'author': authors,
     'tags': tags,
     'summary': summaries,
     'words': words,
     'chapters': chapters,
     'comment': comments,
     'kudos': kudos,
     'bookmarks': bookmarks,
     'hits': hits,
})
print(fanfic_test_df.info())
fanfic_test_df.tail(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      40 non-null     object 
 1   author     40 non-null     object 
 2   tags       40 non-null     object 
 3   summary    40 non-null     object 
 4   words      40 non-null     float64
 5   chapters   40 non-null     int64  
 6   comment    40 non-null     int64  
 7   kudos      40 non-null     int64  
 8   bookmarks  40 non-null     int64  
 9   hits       40 non-null     int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 3.2+ KB
None


,title,author,tags,summary,words,chapters,comment,kudos,bookmarks,hits
38,A Street in a Strange World,Giddygeek,"[Good Omens - Neil Gaiman & Terry Pratchett, G...",“How did you ever know?” Aziraphale asked Crow...,8238.0,1,206,5927,1196,39934
39,Lie Back and Think of Dinner,jessthereckless,"[Good Omens - Neil Gaiman & Terry Pratchett, G...","""Crowley, this is a disaster. This is everythi...",11014.0,1,399,5854,1477,65120


In [162]:
def get_last_index(id_):
    url = "https://archiveofourown.org/works/" + str(id_) + "/kudos?before=4577931054&page=1"
    response = get(url)
    last_index_container = html_soup.find('ol', class_ = "pagination actions")
    index_list = last_index_container.find_all('li')
    last_index = index_list[-2].get_text(strip=True)
    return last_index

In [163]:
def get_kudos(id_):
    last_page_index = get_last_index(id_)
    kudos = [] 
    for i in range(1, int(last_page_index)):
        working_url = "https://archiveofourown.org/works/" + str(id_) + "/kudos?before=4577931054&page=" + str(i)
        response = get(working_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        kudos_container = html_soup.find('div', class_ = "kudos-index region")
        if type(kudos_container).__name__ == 'NoneType':
            continue
        kudos_div = kudos_container.find('p', class_="kudos")
        if type(kudos_div).__name__ != 'NoneType':
            kudos_people = kudos_div.find_all('a')
            for kudos_person in kudos_people:
                person = kudos_person.get_text(strip=True)
                kudos.append(person)
    
    return kudos

In [164]:
# make a method that browse the pages and populate the lists

def populate_Dataframe(limit):
    ids = [] 
    titles = []
    authors = []
    tags = []
    summaries = [] 
    words = [] 
    chapters = [] 
    comments = [] 
    kudos = [] 
    bookmarks = [] 
    hits = []
    kudos_users = []
    fiction_ids = []
    
    for i in range(1, limit):
        url = "https://archiveofourown.org/works/search?commit=Search&page=" + str(i) + "&work_search%5Bbookmarks_count%5D=&work_search%5Bcharacter_names%5D=&work_search%5Bcomments_count%5D=&work_search%5Bcomplete%5D=&work_search%5Bcreators%5D=&work_search%5Bcrossover%5D=&work_search%5Bfandom_names%5D=Good+Omens+%28TV%29%2CGood+Omens+-+Neil+Gaiman+%26+Terry+Pratchett&work_search%5Bfreeform_names%5D=&work_search%5Bhits%5D=&work_search%5Bkudos_count%5D=%26gt%3B100&work_search%5Blanguage_id%5D=&work_search%5Bquery%5D=&work_search%5Brating_ids%5D=&work_search%5Brelationship_names%5D=&work_search%5Brevised_at%5D=&work_search%5Bsingle_chapter%5D=0&work_search%5Bsort_column%5D=kudos_count&work_search%5Bsort_direction%5D=desc&work_search%5Btitle%5D=&work_search%5Bword_count%5D="
        response = get(url)
        if response.status_code != 200:
            pass
        html_soup = BeautifulSoup(response.text, 'html.parser')
        statement_containers = html_soup.find_all('li')
        fiction_works = []
        for s in statement_containers:
            if s.get('role') != None:
                fiction_works.append(s)
        
        for fiction_work in fiction_works:
            get_info(fiction_work, titles, authors, tags, summaries, words, chapters, comments, kudos, bookmarks, hits, fiction_ids)
    
    print(len(titles))
    print(len(authors))
    print(len(tags))
    print(len(summaries))
    print(len(words))
    print(len(chapters))
    print(len(comments))
    print(len(kudos))
    print(len(bookmarks))
    print(len(hits))
    fanfic_df = pd.DataFrame(
    {'id' : fiction_ids,
     'title': titles,
     'author': authors,
     'tags': tags,
     'summary': summaries,
     'words': words,
     'chapters': chapters,
     'comment': comments,
     'kudos': kudos,
     'bookmarks': bookmarks,
     'hits': hits,
     })
    
    return fanfic_df
    

In [165]:
df = populate_Dataframe(3)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

In [166]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         40 non-null     int64  
 1   title      40 non-null     object 
 2   author     40 non-null     object 
 3   tags       40 non-null     object 
 4   summary    40 non-null     object 
 5   words      40 non-null     float64
 6   chapters   40 non-null     int64  
 7   comment    40 non-null     int64  
 8   kudos      40 non-null     int64  
 9   bookmarks  40 non-null     int64  
 10  hits       40 non-null     int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 3.6+ KB


,id,title,author,tags,summary,words,chapters,comment,kudos,bookmarks,hits
0,19074079,In the (Second) Beginning,cherryfeather,"[Good Omens (TV), Good Omens - Neil Gaiman & T...",As their lunch stretches on Aziraphale slowly ...,2661.0,1,565,25409,3999,151598
1,19153375,A Home at the Beginning of the World,stereobone,"[Good Omens (TV), Good Omens - Neil Gaiman & T...","""Oh,"" Aziraphale says. ""I think Crowley might ...",5867.0,1,700,24785,4876,179338
2,19874908,"You, Soft and Only",thehoyden,"[Good Omens (TV), Good Omens - Neil Gaiman & T...",He hadn’t expected a sudden lapful of angel.“V...,9400.0,1,882,17833,4505,156891
3,17752469,such surpassing brightness,bibliocratic,"[Good Omens - Neil Gaiman & Terry Pratchett, G...",The revelation that Aziraphale might have been...,7733.0,1,722,15523,3316,145689
4,19229686,An Invitation You Can't Decline,thehoyden,"[Good Omens - Neil Gaiman & Terry Pratchett, G...","“I have standards,” Aziraphale huffed.“Don’t I...",1963.0,1,510,14644,2341,114240


In [171]:
df['id']
df['id'].to_csv('fanfic_ids.csv')

In [172]:
print(df['id'])

0     19074079
1     19153375
2     19874908
3     17752469
4     19229686
5      1344454
6     19092469
7     19386730
8     19306813
9     19108483
10    19132273
11    19249492
12    19468402
13    17804114
14    21060230
15    19802275
16    19232359
17    19246183
18    19476511
19    19103938
20     1914000
21    19724044
22    18993916
23    19237858
24    20535329
25    19073335
26    19114750
27    19322800
28    19160302
29    19870036
30    19893115
31    18043838
32    19150996
33    20362381
34    19378126
35    19067029
36    18949555
37    20480360
38    19190599
39    19136509
Name: id, dtype: int64


# To be run to get the data from all the pages

In [169]:
final_fanfic_df = populate_Dataframe(650)
# save dataframe to csv
final_fanfic_df.to_csv('fanfic_final.csv')

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'NoneType'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [170]:
final_fanfic_df.tail()

NameError: name 'final_fanfic_df' is not defined